### 3. SVD of Quantum Thermal States (16 points)

Consider the HDF5 file provided in Canvas for this exercise, `ThermalMatrices.h5`. This file contains two matrices, `T10` and `T100`, of dimension 2048 × 2048. The matrices represent the quantum states (density matrices) of a system of 11 spins, at temperatures \( T = 10J \) and \( T = 100J \) respectively (\( J \) is the hopping energy in the system). Please provide the solution of the following problems in a Jupyter Notebook.

#### (a) Calculate the SVD of both matrices, and plot their normalized singular values in the same figure (4 points).


In [29]:
using ITensors          
using Plots             
using LinearAlgebra    
using HDF5              


In [ ]:
# Load the matrices from the HDF5 file
file = h5open("ThermalMatrices.h5", "r")
T10 = read(file, "T10")
T100 = read(file, "T100")
close(file)

# Calculate the SVD
U10, S10, V10 = svd(T10)
U100, S100, V100 = svd(T100)

# Normalize the singular values
S10_norm = S10 / sqrt(sum(S10.^2))
S100_norm = S100 / sqrt(sum(S100.^2))

# Plot the normalized singular values
plot(1:length(S10_norm), S10_norm, label="T10", xlabel="Index", ylabel="Normalized Singular Value",xscale =:log10 ,title="Normalized Singular Values")
plot!(1:length(S100_norm), S100_norm,xscale = :log10,label="T100")


#### (b) Compress both matrices so the truncation error is approximately 1%. How many singular values need to be kept for each matrix to achieve this? (4 points)


In [ ]:
function num_to_keep(S, error_threshold)
    cumulative_sum = cumsum(S)
    total_sum = sqrt(sum(S.^2))
    for i in 1:length(S)
        if cumulative_sum[i] / total_sum >= (1 - error_threshold)
            return i
        end
    end
    return length(S)
end

#  number to keep for 1%  error
error_threshold = 0.01
num_T10 = num_to_keep(S10_norm, error_threshold)
num_T100 = num_to_keep(S100_norm, error_threshold)

println("Number of singular values to keep for T10: ", num_T10)
println("Number of singular values to keep for T100: ", num_T100)

# Truncate the matrices
T10_truncated = U10[:, 1:num_T10] * Diagonal(S10[1:num_T10]) * V10[:, 1:num_T10]';
T100_truncated = U100[:, 1:num_T100] * Diagonal(S100[1:num_T100]) * V100[:, 1:num_T100]';

#### (c) Compare the eigenvalues of `T10` with those of its truncated version (4 points).


In [ ]:
eigvals_T10 = eigvals(T10)
eigvals_T10_truncated = eigvals(T10_truncated)

# Plot the eigenvalues
plot(1:length(eigvals_T10), sort(real(eigvals_T10), rev=true), label="T10", xlabel="Index", ylabel="Eigenvalue",title="Eigenvalues Comparison",xscale = :log10)
plot!(1:length(eigvals_T10_truncated), sort(real(eigvals_T10_truncated),rev=true), label="T10 Truncated",xscale = :log10)

#### (d) Calculate the von Neumann entropy of the matrices `T10` and `T100` (4 points).

In [ ]:

# Calculate the von Neumann entropy
entropy_T10 =  -sum(eigvals(T10) .* log.(eigvals(T10)));
entropy_T100 = -sum(eigvals(T100) .* log.(eigvals(T100)));

println("Von Neumann entropy of T10: ", entropy_T10)
println("Von Neumann entropy of T100: ", entropy_T100)

### 4. Polar Decomposition (12 points)

The polar decomposition was recently put forward as an alternative to SVD to implement MPS structures on tensor processing units; see Ganahl et al, *PRX Quantum* 4, 010317 (2023). In this decomposition, a matrix \( O \) of dimensions \( m X n \) is written as the product of an isometric \( m X n \) matrix \( P \) and a positive semi-definite Hermitian matrix  H of dimension \( n X n \): 

 O=P*H 

Please provide the solution of the following problems in a Jupyter Notebook.

#### (a) Implement the iterative algorithm described in Appendix A of the article, and apply it to the matrix `T10` of Exercise 3 to obtain its polar decomposition (8 points).




In [34]:
using ITensors          
using Plots             
using LinearAlgebra    
using HDF5 

In [ ]:
# define polar_decomposition function as paper
function polar_factor(M)
    z = norm(M)  # Frobenius norm
    P = M / z    # normalized P
    converged = false #start with false
    
    iter = 0
    maxiter=50
    while !converged && iter < maxiter
        iter += 1
        T = P' * P; # T as in the algrithm
        
        P_new = (3/2) * P - (1/2) * (P * T) #iteration
        
        
        converged = check_unitarity(P_new) #Check convergence
        
        P = P_new
    end
    print("Converged after $iter iterations ")
    # Return the result
    return P, P' * M
end

function check_unitarity(M)
    # Check if M is unitary (M*M' ≈ I for rectangular matrices)
    
    product = M * M'
    error = norm(M * M' - I)
    tolerance = 1e-6  # error
    return error < tolerance
end


##### Reload T10 for polar decomposition

In [36]:
file = h5open("ThermalMatrices.h5", "r");
T10 = read(file, "T10");


##### Polar deposition

In [ ]:
@time P_i, H_i = polar_factor(T10);


In [ ]:
@show size(P_i) #show P
@show size(H_i) #show H

In [ ]:
O_reconstructed = P_i * H_i
reconstruction_error = norm(T10 - O_reconstructed) / norm(T10)
println("Reconstruction error: ", reconstruction_error)


#### (b) Compare the matrices \( P \) and \( H \) obtained in (a) with those obtained from an SVD of \( O \), using the Frobenius norm of their differences (4 points).

In [ ]:
@time U10, Σ, V10 = svd(T10) #svd method
P_svd=U10*V10'; # svd P
H_svd = V10 * Diagonal(Σ) * V10'; #svd H

P_diff = norm(P_i - P_svd)
H_diff = norm(H_i - H_svd)

# Compute the relative differences
P_diff_rel = P_diff / norm(P_svd)
H_diff_rel = H_diff / norm(H_svd)

println("Relative Frobenius norm difference for P: ", P_diff_rel)
println("Relative Frobenius norm difference for H: ", H_diff_rel)
#@show H_diff_rel

# 5. Color Image Compression (14 points)

Please provide the solution of the following problems in a Jupyter Notebook.

#### a) Implement a code to compress a color image of your preference using SVD, fixing the number of maintained singular values or the truncation error. Do this with the LinearAlgebra package of Julia. *Hint*: Perform the compression of the matrices of each color channel independently, and then combine the results (8 points).




**Import packages**

In [41]:
using LinearAlgebra
using Plots
using Images
using ITensors

**Load photos**

In [ ]:
Photo = load("stack14.png");
print(size(Photo))
p1=plot(RGB.(Photo),title="original")
display(p1)



**Display singular values**

In [ ]:
PhotoSVD = svd(Gray.(Photo));
plot(PhotoSVD.S, xaxis=:log, yaxis=:log,label="singular value",xlabel="Index", ylabel="Eigenvalue",title="Eigenvalues")

***'<big>'How many singular values will be kept***

In [44]:
χ = 10;

**Compressing images in all three channels**

In [ ]:

img_CHW = channelview(Photo) 
pho_HWC = permutedims(img_CHW, (2, 3, 1))# conver matirx 
Compressed= zeros(size(pho_HWC))
for i = 1:3 #three color channel
    img=pho_HWC[:,:,i];
    PhotoSVD = svd(Gray.(img));
    n = size(PhotoSVD.S,1)
    rem = n-χ;
    # Keep only χ singular values
    Sapprox=Diagonal(copy(PhotoSVD.S));
    for k = 1:rem
        Sapprox[n+1-k,n+1-k]=0;
    end
    Compressed[:,:,i] = PhotoSVD.U*Sapprox*PhotoSVD.Vt;
end
Compressed_rgb = permutedims(Compressed, (3, 1, 2)) # change matrix to photo

# Create RGB image and plot
com_rgb = colorview(RGB, Compressed_rgb)
plot(com_rgb,title="After Compressed")

#### b) Implement a code for the same task, using the SVD function of ITensor (6 points).

#### Load images ####

In [ ]:
Photo = load("stack14.png");
#print(size(Photo));
p1=plot(RGB.(Photo),title="original");
display(p1);

img_CHW = channelview(Photo) ;
pho_HWC = permutedims(img_CHW, (2, 3, 1));
pho_size=size(pho_HWC)

#### Initialize index

In [ ]:
i=Index(pho_size[1]);
j=Index(pho_size[2]);
k=Index(pho_size[3]);
@show i,j,k

#### SVD with merge i,k legs

In [ ]:
P=ITensor(pho_HWC,i,j,k);#matrix to ITensor

U,S,V=svd(P,(i,k),cutoff=1E-10,maxdim=10); #compress with dim of 10
truncerr= (norm(U*S*V-P)/norm(P))^2;
@show  truncerr #show err
#@show inds(S)

#### Plot singular values

In [49]:
# find shared legs
s1 = commonind(U, S);
s2=  commonind(V, S);

In [ ]:
ss=Array(S,s1,s2);
size(ss)
plot(diag(ss), yaxis=:log,label="singular value",xlabel="Index", ylabel="Eigenvalue",title="Eigenvalues after Truncated")

#### Plot compressed photo

In [ ]:
P_c=U*S*V;
P_new=Array(P_c,i,j,k);#ITensor to array
@show size(P_new)

In [ ]:
Compressed_rgb = permutedims(P_new, (3, 1, 2))

# Create RGB image and plot
com_rgb = colorview(RGB, Compressed_rgb)
plot(com_rgb,title="After Compressed")

# 8. 2D function as MPS (16 points)

Please provide the solution of the following problems in a Jupyter Notebook.

#### a) Encode the function f(x, y) = sin(x + y) * cos(xy) as an MPS, in the range x ∈ [0, π] and y ∈ [0, 2π]. Use N = 7 qubits to define the spatial grid in each dimension (6 points). 





In [53]:
using ITensors          
using Plots             
using LinearAlgebra     
using HDF5              


In [ ]:
    
    N = 7 ; # 7 qubit sites
    num = 2^N;
        
        # Define the x and y grids
    x_vals = range(0, π, length=num);
    y_vals = range(0, 2π, length=num);
     # Define the function
    f(x, y) = sin(x + y) * cos(x * y)
    
    F = [f(x, y) for x in x_vals, y in y_vals]
    # plot Original function
    p1 = heatmap(x_vals, y_vals, F, 
             title="Original Function", 
             xlabel="x", ylabel="y")
    

In [ ]:
ITensors.disable_warn_order()
sites = siteinds("Qubit", 2*N);
A_F=reshape(F,num^2);# 2d matrix to array
M = MPS(A_F,sites);#array to MPS
@show M

#### b) What is the maximal bond dimension of the MPS with no truncation? (4 points)


In [ ]:
# Extract bond dimensions of the MPS
bond_dims = [dim(linkind(M, n)) for n in 1:length(M)-1]
println("Bond dimensions without truncation: ", bond_dims)

# The maximal bond dimension is:
max_bond_dim = maximum(bond_dims)
println("Maximal bond dimension without truncation: ", max_bond_dim)


#### c) Make a figure of the function (3D surface or contour) when truncating to bond dimension χ = 3, and compare with the original function (6 points).

In [57]:
# Desired bond dimension
χ = 3;

# Truncate the MPS to the desired bond dimension
psi_trunc = truncate(M; maxdim=χ);

In [ ]:
bond_dims = [dim(linkind(psi_trunc , n)) for n in 1:length(psi_trunc )-1]
println("Bond dimensions after truncation: ", bond_dims)

# The maximal bond dimension is:
max_bond_dim = maximum(bond_dims)
println("Maximal bond dimension after truncation: ", max_bond_dim)

In [59]:
#contract mps back to tensor
ITensors.disable_warn_order()
psi_tensor = ITensor(1.)
for i = 1:2*N
    psi_tensor *= psi_trunc[i]
end

In [60]:
f_approx_vec=Array(psi_tensor,sites);#tensor to array
f_approx = reshape(f_approx_vec, num, num);#array back to 2D matrix

In [ ]:
p1 = contour(x_vals, y_vals, F, 
             title="Original Function", 
             xlabel="x", ylabel="y", 
            )
display(p1)
p2 = contour(x_vals, y_vals, f_approx, 
            title="Approximated Function(χ=3)",
             xlabel="x", ylabel="y", 
             )
display(p2)
